In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import opendatasets as od
import pandas as pd
from PIL import Image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
od.download("https://www.kaggle.com/datasets/gonzajl/tumores-cerebrales-mri-dataset")
datos = pd.read_csv("./tumores-cerebrales-mri-dataset/datos.csv")

Skipping, found downloaded files in "./tumores-cerebrales-mri-dataset" (use force=True to force download)


In [ ]:
cant_max = 9500

glioma_df = datos[datos['glioma'] == 1].head(cant_max)
meningioma_df = datos[datos['meningioma'] == 1].head(cant_max)
pituitary_df = datos[datos['pituitary'] == 1].head(cant_max)
no_tumor_df = datos[datos['no_tumor'] == 1].head(cant_max)

datos = pd.concat([glioma_df, meningioma_df, pituitary_df, no_tumor_df], ignore_index=True)

In [ ]:
datos = datos.sample(frac=1).reset_index(drop=True)

In [ ]:
def cargar_imagenes(nombres, et):
    imagenes = []
    etiquetas = []
    for i in range(len(nombres)):
        ruta = 'tumores-cerebrales-mri-dataset/' + nombres[i][2:]
        imagen = cv2.imread(ruta, cv2.IMREAD_COLOR)
        imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
        imagen = imagen.reshape(imagen.shape[0], imagen.shape[0], 1)
        imagenes.append(imagen)
        etiquetas.append(list(et.iloc[i]))

    return imagenes, etiquetas

In [ ]:
paths = datos.iloc[:, 0]
tags = datos.iloc[:, 1:]   #ignoro filas particulares y traigo las columnas de 1 a fin

In [ ]:
imagenes, etiquetas = cargar_imagenes(paths, tags)

In [ ]:
div_test = int(len(imagenes) * 0.8)

In [ ]:
imagenes_entrenamiento, imagenes_prueba = imagenes[:div_test], imagenes[div_test:]
etiquetas_entrenamiento, etiquetas_prueba = etiquetas[:div_test], etiquetas[div_test:]

In [ ]:
print(len(imagenes_entrenamiento)) #esto no es necesario pero muestra la cantidad de elementos en cada array.
print(len(imagenes_prueba))

print(len(etiquetas_entrenamiento))
print(len(etiquetas_prueba))

30400
7600
30400
7600


In [ ]:
imagenes_entrenamiento = np.array(imagenes_entrenamiento).astype(float) / 255
imagenes_prueba = np.array(imagenes_prueba).astype(float) / 255

etiquetas_entrenamiento = np.array(etiquetas_entrenamiento)
etiquetas_prueba = np.array(etiquetas_prueba)

#---

forma_etiqueta = etiquetas_entrenamiento[0].shape #Esto muestra el formato actual de las etiquetas, en particular muestra la primera
                                                    #del array de entrenamiento
print(forma_etiqueta)

(4,)


In [ ]:
modelo_cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(224, 224, 1)),
    tf.keras.layers.MaxPooling2D(3, 3),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(4, activation='sigmoid')
])

In [ ]:
modelo_cnn.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

In [ ]:
print("Entrenando modelo convolucional...")
historial_cnn = modelo_cnn.fit(imagenes_entrenamiento, etiquetas_entrenamiento, epochs=10, validation_data=(imagenes_prueba, etiquetas_prueba), use_multiprocessing=True, shuffle=True)
print("Modelo convolucional entrenado!")

Entrenando modelo convolucional...
Epoch 1/10


2023-10-05 18:36:12.971958: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 6101401600 exceeds 10% of free system memory.


950/950 [==============================] - 190s 200ms/step - loss: 0.0346 - accuracy: 0.9764 - val_loss: 0.0687 - val_accuracy: 0.9613
Epoch 2/10
950/950 [==============================] - 190s 200ms/step - loss: 0.0334 - accuracy: 0.9763 - val_loss: 0.0704 - val_accuracy: 0.9650
Epoch 3/10
950/950 [==============================] - 188s 198ms/step - loss: 0.0322 - accuracy: 0.9782 - val_loss: 0.0720 - val_accuracy: 0.9626
Epoch 4/10
950/950 [==============================] - 188s 198ms/step - loss: 0.0291 - accuracy: 0.9809 - val_loss: 0.0990 - val_accuracy: 0.9438
Epoch 5/10
950/950 [==============================] - 187s 197ms/step - loss: 0.0273 - accuracy: 0.9815 - val_loss: 0.0742 - val_accuracy: 0.9607
Epoch 6/10
950/950 [==============================] - 190s 200ms/step - loss: 0.0265 - accuracy: 0.9820 - val_loss: 0.0764 - val_accuracy: 0.9628
Epoch 7/10
950/950 [==============================] - 187s 197ms/step - loss: 0.0238 - accuracy: 0.9839 - val_loss: 0.0830 - val_accura

In [ ]:
def es_correcta(prediccion, esperado):
    return prediccion.index(max(prediccion)) == esperado.index(max(esperado))

In [ ]:
correctas_segun_tipo = [0, 0, 0, 0]
falladas_segun_tipo = [0, 0, 0, 0]

predicciones = modelo_cnn.predict(imagenes_prueba)
print(f"Cantidad de predicciones: {len(predicciones)}")

for i in range(len(predicciones)):
    prediccion = list(predicciones[i])
    index = prediccion.index(max(prediccion))

    if es_correcta(prediccion, list(etiquetas_prueba[i])):
        correctas_segun_tipo[index] += 1
    else:
        falladas_segun_tipo[index] += 1

cant_totales = list(map(lambda x, y: x + y, correctas_segun_tipo, falladas_segun_tipo))
print("Etiquetas:   [G,  M,  P,  N]")
print(f"Total:       {cant_totales}")
print(f"Correctas:   {correctas_segun_tipo}")
print(f"Incorrectas: {falladas_segun_tipo}")

238/238 [==============================] - 13s 56ms/step
Cantidad de predicciones: 7600
Etiquetas:   [G,  M,  P,  N]
Total:       [1920, 1893, 1868, 1919]
Correctas:   [1838, 1808, 1845, 1857]
Incorrectas: [82, 85, 23, 62]


In [ ]:
!mkdir carpeta_salida

modelo_cnn.save("carpeta_salida/deteccion-tumores-cnn.h5")

mkdir: cannot create directory ‘carpeta_salida’: File exists


In [ ]:
!pip install tensorflowjs

In [ ]:
!tensorflowjs_converter --input_format keras carpeta_salida/deteccion-tumores-cnn.h5 carpeta_salida

2023-10-05 19:43:58.957594: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-05 19:43:58.999119: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-05 19:43:58.999162: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-05 19:43:58.999193: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-05 19:43:59.006678: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-05 19:43:59.006886: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens